# Submission Demo Notebook

End-to-end demonstration of NorthStar triage & analytics pipeline.

Phases covered: ingest (multimodal), embeddings refresh, freeform + ticket triage, dashboard views creation.

This notebook is idempotent: safe to re-run; it uses minimal sample data.

## 1. Environment Setup
Set required environment variables (replace with your project).

In [ ]:
import os, json, textwrap, pathlib, sys
print('Python', sys.version)
os.environ.setdefault('PROJECT_ID','bq_project_northstar')
os.environ.setdefault('DATASET','demo_ai')
os.environ.setdefault('LOCATION','US')
os.environ['BIGQUERY_REAL']='1'
for k in ['PROJECT_ID','DATASET','LOCATION','BIGQUERY_REAL']:
    print(k,'=',os.environ.get(k))

## 2. Install Dependencies
If running in a constrained environment (e.g., Kaggle), you can trim extras.

Uncomment and run if needed.

In [ ]:
# !pip install -e .[bigquery,ingest,dashboard] -q

## 3. Preflight Models (optional)
Validates remote model presence if environment variables (BQ_EMBED_MODEL/BQ_GEN_MODEL) are set.

In [ ]:
!python scripts/check_bq_resources.py --models-only || echo 'Preflight models check failed (ensure models exist or create them).'

## 4. Create Views
Idempotent creation of analytics views.

In [ ]:
!python scripts/create_views.py

## 5. Ingest Samples
Processes sample files (logs / pdf / images) and refreshes embeddings in a loop.

In [ ]:
!python -m core.cli ingest --path samples --type auto --max-tokens 512 --refresh-loop

## 6. Freeform Triage
Generates a playbook for an ad-hoc issue.

In [ ]:
!python -m core.cli triage --title '500 after reset' --body 'android camera' --severity P1 --out out/nb_freeform.md

## 7. Ticket Triage
Attempts to triage DEMO-1 if present (non-fatal if absent).

In [ ]:
import subprocess, shlex, os, pathlib
ticket_out = pathlib.Path('out/nb_ticket.md')
cmd = 'python -m core.cli triage --ticket-id DEMO-1 --severity P1 --out out/nb_ticket.md --max-comments 3'
res = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print(res.stdout)
if res.returncode!=0:
    print('Ticket triage skipped or failed (OK for demo).')

## 8. Display Outputs
Show first 40 lines of generated playbooks.

In [ ]:
from pathlib import Path
def head(p: str, n=40):
    path=Path(p)
    if not path.exists():
        return f'Missing: {p}'
    return '
'.join(path.read_text(encoding='utf-8').splitlines()[:n])
print('--- Freeform ---')
print(head('out/nb_freeform.md'))
print('
--- Ticket ---')
print(head('out/nb_ticket.md'))

## 9. Dashboard Note
Run locally (not inside hosted notebook) to explore analytics:

```bash
make dashboard
```


## 10. Safety
Playbooks enforce basic verifier checks; snippets truncated & masked; no destructive actions executed.